In [1]:
import os, sys
import time
import copy
import random
import numpy as np
from sklearn.utils import shuffle

In [2]:
with open("llvmpasses.txt", mode="r") as f:
    PASSES = [x.strip() for x in f.readlines()]

In [3]:
PASSES[:5]

['-loop-unswitch',
 '-postdomtree',
 '-opt-remark-emitter',
 '-forceattrs',
 '-float2int']

In [4]:
pass_dir = {}
for i in range(0, len(PASSES)):
    pass_dir[i] = PASSES[i]

In [5]:
pass_dir

{0: '-loop-unswitch',
 1: '-postdomtree',
 2: '-opt-remark-emitter',
 3: '-forceattrs',
 4: '-float2int',
 5: '-loop-accesses',
 6: '-loop-simplify',
 7: '-memdep',
 8: '-scoped-noalias',
 9: '-memcpyopt',
 10: '-callsite-splitting',
 11: '-gvn',
 12: '-globaldce',
 13: '-targetlibinfo',
 14: '-loop-idiom',
 15: '-aa',
 16: '-attributor',
 17: '-lower-constant-intrinsics',
 18: '-globals-aa',
 19: '-demanded-bits',
 20: '-transform-warning',
 21: '-aggressive-instcombine',
 22: '-correlated-propagation',
 23: '-lcssa',
 24: '-memoryssa',
 25: '-early-cse',
 26: '-deadargelim',
 27: '-ipsccp',
 28: '-lazy-block-freq',
 29: '-licm',
 30: '-jump-threading',
 31: '-tbaa',
 32: '-tailcallelim',
 33: '-loop-rotate',
 34: '-slp-vectorizer',
 35: '-sroa',
 36: '-basicaa',
 37: '-barrier',
 38: '-pgo-memop-opt',
 39: '-instcombine',
 40: '-elim-avail-extern',
 41: '-globalopt',
 42: '-loop-deletion',
 43: '-loop-load-elim',
 44: '-prune-eh',
 45: '-loops',
 46: '-libcalls-shrinkwrap',
 47: '-la

In [7]:
option = 0
for iter in range(5):
    rand_passes = shuffle(PASSES)
    if option == 1:
        rand_passes = [x[1:] for x in rand_passes]
        passes_str = f'-passes="{",".join(rand_passes)}"'
    else:
        passes_str = " ".join(rand_passes)

In [8]:
passes_str

'-loop-sink -bdce -inferattrs -scoped-noalias -verify -instsimplify -aa -indvars -loop-unroll -ee-instrument -loops -jump-threading -lazy-value-info -argpromotion -lcssa-verification -tailcallelim -targetlibinfo -loop-simplify -loop-distribute -forceattrs -branch-prob -postdomtree -licm -lower-constant-intrinsics -early-cse -rpo-functionattrs -deadargelim -ipsccp -strip-dead-prototypes -alignment-from-assumptions -loop-load-elim -dse -gvn -scalar-evolution -opt-remark-emitter -memdep -globals-aa -constmerge -tbaa -block-freq -sccp -speculative-execution -called-value-propagation -float2int -loop-unswitch -basiccg -prune-eh -sroa -loop-rotate -adce -lazy-block-freq -lcssa -domtree -demanded-bits -callsite-splitting -basicaa -assumption-cache-tracker -elim-avail-extern -loop-vectorize -memcpyopt -early-cse-memssa -loop-deletion -tti -slp-vectorizer -div-rem-pairs -loop-idiom -reassociate -libcalls-shrinkwrap -lazy-branch-prob -mem2reg -attributor -functionattrs -aggressive-instcombine -b

In [14]:
pg_path = "polybench/datamining/correlation/correlation/correlation.c"
pg_name = "correlation"

In [15]:
ini_cmd1 = 'clang -O0 -emit-llvm -I ../polybench/utilities -I ../polybench/datamining/correlation/correlation -c {path} -o {name}.bc'
ini_cmd2 = 'clang -O0 -emit-llvm -I ../polybench/utilities -I ../polybench/datamining/correlation/correlation -c ../polybench/utilities/polybench.c -o polybench.bc;'
cmd1 = ' -S polybench.bc -o polybench.opt.bc; llc -O0 -filetype=obj polybench.opt.bc -o polybench.o'
cmd2 = ' -S {name}.bc -o {name}.opt.bc; llc -O0 -filetype=obj {name}.opt.bc -o {name}.o'
cmd3 = 'clang -O0 -no-pie -lm *.o -o {name}'
cmd4 = './{name}'
clean_cmd = 'rm -rf *bc *.o *.I *.S {name}'

In [16]:
for iter in range(10):
    res = []
    std_time = []
    Median_time = []
    Mean_time = []
    Min_time = []
    Max_time = []
    rand_passes = shuffle(PASSES)
    print(f"Iteration {iter + 1}:", " ".join(rand_passes))
    print("*" * 5)
    os.system(clean_cmd.format(name=pg_name))
    os.system(ini_cmd1.format(path=pg_path, name=pg_name))
    os.system(ini_cmd2)
    os.system("opt -O0 " + " ".join(rand_passes) + cmd1)
    os.system("opt -O0 " + " ".join(rand_passes) + cmd2.format(name=pg_name))
    print(cmd3.format(name=pg_name))
    if os.system(cmd3.format(name=pg_name)) == 0:
        print("Compiled! Measuring execution time...")
    exe_times = []
    for rtime in range(10):
        begin = time.time()
        ret = os.system(cmd4.format(name=pg_name))
        if ret > 0:
            continue
        end = time.time()
        print(f"R{rtime + 1}:", end - begin)
        exe_times.append(end - begin)
    print(exe_times)
    print(f"Std: {np.std(exe_times)}; Median: {np.median(exe_times)}; Mean: {np.mean(exe_times)}; Min: {np.min(exe_times)}; Max: {np.max(exe_times)}")
    print("=" * 20)
    res.append(exe_times)
    std_time.append(np.std(exe_times))
    Median_time.append(np.median(exe_times))
    Mean_time.append(np.mean(exe_times))
    Min_time.append(np.min(exe_times))
    Max_time.append(np.max(exe_times))

Iteration 1: -tti -lazy-value-info -div-rem-pairs -postdomtree -phi-values -correlated-propagation -instcombine -barrier -verify -rpo-functionattrs -globalopt -memcpyopt -sroa -jump-threading -licm -loop-load-elim -loop-rotate -loop-distribute -globals-aa -memdep -loops -instsimplify -lower-constant-intrinsics -early-cse-memssa -ipsccp -gvn -forceattrs -lazy-branch-prob -simplifycfg -demanded-bits -loop-unroll -opt-remark-emitter -lower-expect -profile-summary-info -loop-idiom -tbaa -float2int -reassociate -attributor -callsite-splitting -adce -alignment-from-assumptions -loop-unswitch -early-cse -tailcallelim -lcssa-verification -dse -aggressive-instcombine -argpromotion -basiccg -elim-avail-extern -slp-vectorizer -loop-deletion -targetlibinfo -ee-instrument -loop-simplify -assumption-cache-tracker -lcssa -transform-warning -memoryssa -domtree -mldst-motion -inferattrs -mem2reg -constmerge -indvars -pgo-memop-opt -scalar-evolution -lazy-block-freq -libcalls-shrinkwrap -loop-sink -sccp

C:\Users\LYue7\anaconda3\envs\autotuning\lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\LYue7\anaconda3\envs\autotuning\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\LYue7\anaconda3\envs\autotuning\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\LYue7\anaconda3\envs\autotuning\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\LYue7\anaconda3\envs\autotuning\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: zero-size array to reduction operation minimum which has no identity